<a href="https://colab.research.google.com/github/hemantkashniyal/EVA4/blob/master/notebooks/EVA4_Session_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1)     # input:28x28x1, output:28x28x8, RF: 3
        self.bn1 = nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8, 16, 3, padding=1)    # input:28x28x8, output:28x28x16, RF: 5
        self.bn2 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(2, 2)                # input:28x28x16, output:14x14x16, RF: 10 *MP
        self.dp1 = nn.Dropout2d(0.2)

        self.conv3 = nn.Conv2d(16, 32, 3, padding=1)   # input:14x14x16, output:14x14x32, RF: 12
        self.bn3 = nn.BatchNorm2d(32)
        self.conv4 = nn.Conv2d(32, 8, 3, padding=1)    # input:14x14x32, output:14x14x8, RF: 14
        self.bn4 = nn.BatchNorm2d(8)
        self.dp4 = nn.Dropout2d(0.1)
        self.pool2 = nn.MaxPool2d(2, 2)                # input:14x14x8, output:7x7x8, RF: 28 *MP
        self.dp2 = nn.Dropout2d(0.2)

        self.conv5 = nn.Conv2d(8, 16, 3)               # input7x7x8, output:5x5x16, RF: 30
        self.bn5 = nn.BatchNorm2d(16)
        self.dp5 = nn.Dropout2d(0.1)
        self.conv6 = nn.Conv2d(16, 32, 3)              # input5x5x16, output:3x3x32, RF: 32
        self.bn6 = nn.BatchNorm2d(32)
        self.dp6 = nn.Dropout2d(0.1)
        self.conv7 = nn.Conv2d(32, 10, 3)              # input3x3x32, output:1x1x10, RF: 34
        self.gap = nn.AvgPool2d(1)

    def forward(self, x):
        x =      self.dp1(self.pool1(self.bn2(F.relu(self.conv2(self.bn1(F.relu(self.conv1(x))))))))
        x =      self.dp2(self.pool2(self.bn4(F.relu(self.conv4(self.bn3(F.relu(self.conv3(x))))))))
        x =                   F.relu(self.bn6(F.relu(self.conv6(self.bn5(F.relu(self.conv5(x)))))))
        x = self.gap(self.conv7(x))
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [4]:
#!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
            Conv2d-3           [-1, 16, 28, 28]           1,168
       BatchNorm2d-4           [-1, 16, 28, 28]              32
         MaxPool2d-5           [-1, 16, 14, 14]               0
         Dropout2d-6           [-1, 16, 14, 14]               0
            Conv2d-7           [-1, 32, 14, 14]           4,640
       BatchNorm2d-8           [-1, 32, 14, 14]              64
            Conv2d-9            [-1, 8, 14, 14]           2,312
      BatchNorm2d-10            [-1, 8, 14, 14]              16
        MaxPool2d-11              [-1, 8, 7, 7]               0
        Dropout2d-12              [-1, 8, 7, 7]               0
           Conv2d-13             [-1, 16, 5, 5]           1,168
      BatchNorm2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'epoch={epoch} loss={loss.item()} batch_id={batch_idx}')
    


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return 100. * correct / len(test_loader.dataset)

In [7]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
max_epoch = 20
desired_accuracy = 99.4
for epoch in range(1, max_epoch+1):
    train(model, device, train_loader, optimizer, epoch)
    accuracy = test(model, device, test_loader)

    if accuracy >= desired_accuracy:
      print("Result epoch: {}, Accuracy: {}, Desired: {} ".format(epoch, accuracy, desired_accuracy))
      break



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0502, Accuracy: 9848/10000 (98.4800%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0388, Accuracy: 9878/10000 (98.7800%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0237, Accuracy: 9929/10000 (99.2900%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0203, Accuracy: 9934/10000 (99.3400%)



epoch=16 loss=0.003918786998838186 batch_id=468: 100%|██████████| 469/469 [00:12<00:00, 36.90it/s]



Test set: Average loss: 0.0226, Accuracy: 9940/10000 (99.4000%)

Result epoch: 16, Accuracy: 99.4, Desired: 99.4 
